In [1]:
import traceback
from pymongo import MongoClient
from elasticsearch import Elasticsearch

In [2]:
db = MongoClient('mongodb://127.0.0.1:27017')['Dogforum']

In [3]:
es = Elasticsearch()

In [4]:
for type_name in db.collection_names():
    print(type_name)

dog_stories


In [6]:
for type_name in db.collection_names():
    cursor = db[type_name].find({}, projection={'_id':False})
    _docs = [x for x in cursor]
    for _doc in _docs[:1]:
        print(_doc)

{'board': 'dog-stories', 'url': 'http://www.dogforum.com/dog-stories/lab-thinks-shes-lifeguard-338226/', 'page': 1, 'title': "Lab thinks she's a lifeguard", 'floor': 1, 'author_url': 'http://www.dogforum.com/members/198602/', 'author_name': 'Sthelena', 'post_date': '10-19-2017, 07:34 PM', 'p_date': '20171019193400', 'author_info': {'member_type': 'Senior Member', 'posts': '367', 'mentioned': '31 Post(s)', 'tagged': '0 thread(s)'}, 'id': 'dog-stories.lab-thinks-shes-lifeguard-338226.F1', 'quotation': [], 'content': "  So just a little story we think is neat, thought I'd share. \nMy wifes dog is a black lab, field type, came from good hunting lines, she says, and I believe her\nAnyway, couple of years ago we went camping on a lake in the mountains. Lake is filled by waterfalls, crystal clear water. Like tap water clear. Just an awesome vacation seeing fall leaf change, hiking, swimming, finding waterfalls by boat etc. \nI decided to go swimming, and found a nice spot with some sandy bank

In [ ]:
"post_date": {
              "type": "date",
              "format": "MM-dd-yyyy HH:mm a"
            },
    
              "p_date": {
            "type": "integer"
          },

In [9]:
def index_map(type_name):
    
    text_mapping = {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    }
    
    n_gram = {
      "type": "text",
      "analyzer": "autocomplete",
      "search_analyzer": "autocomplete_search",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    }
    
    _index_mappings = {
      "settings": {
        "analysis": {
          "analyzer": {
            "autocomplete": {
              "tokenizer": "autocomplete",
              "filter": [
                "lowercase"
              ]
            },
            "autocomplete_search": {
              "tokenizer": "lowercase"
            }
          },
          "tokenizer": {
            "autocomplete": {
              "type": "edge_ngram",
              "min_gram": 4,
              "max_gram": 10,
              "token_chars": [
                "letter"
              ]
            }
          }
        }
      },
      "mappings": {
      "article": {
        "properties": {
          "author_info": {
            "properties": {
              "member_type": text_mapping,
              "mentioned": text_mapping,
              "posts": {
                    "type": "integer"
                  },
              "tagged": text_mapping
            }
          },
          "author_name": text_mapping,
          "author_url": text_mapping,
          "board": text_mapping,
          "content": n_gram,
          "floor": {
                "type": "integer"
              },
          "id": text_mapping,
          "page": {
                "type": "integer"
              },
          "post_date": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm"
                },
          "post_stamp": {
                "type": "integer"
              },
          "title": n_gram,
          "quotation" : n_gram,
          "tags" : n_gram,
          "url": text_mapping
        }
       }
      }
    }
    return _index_mappings

In [10]:
if es.indices.exists(index='dogforum') is not True:
    es.indices.create(index='dogforum', body=index_map(type_name))

In [11]:
for type_name in db.collection_names():
    cursor = db[type_name].find({}, projection={'_id':False})
    _docs = [x for x in cursor]

In [ ]:
doc_type=type_name, 

In [12]:
processed = 0
for _doc in _docs[:1000]:
    try:
        es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
        processed += 1
    except:
        traceback.print_exc()
print('Processed: ' + str(processed))

PUT http://localhost:9200/dogforum/article/dog-stories.lab-thinks-shes-lifeguard-338226.F3 [status:400 request:0.003s]
Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/ana

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

...

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

PUT http://localhost:9200/dogforum/article/dog-stories.missing-abandoned-14114.F2 [status:400 request:0.003s]
Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.

Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se

Processed: 582


Traceback (most recent call last):
  File "<ipython-input-12-087b391ebf53>", line 4, in <module>
    es.index(index='dogforum', doc_type='article', id=_doc.get('id').replace('/',''), body=_doc)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/utils.py", line 76, in _wrapped
    return func(*args, params=params, **kwargs)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/client/__init__.py", line 300, in index
    _make_path(index, doc_type, id), params=params, body=body)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/transport.py", line 314, in perform_request
    status, headers_response, data = connection.perform_request(method, url, params, body, headers=headers, ignore=ignore, timeout=timeout)
  File "/home/iii/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 163, in perform_request
    se